# UserProxyAgent \#
#### The UserProxyAgent is a special built-in agent that acts as a proxy for a user to provide feedback to the team.

#### To use the UserProxyAgent, you can create an instance of it and include it in the team before running the team. The team will decide when to call the UserProxyAgent to ask for feedback from the user.

#### When team calls the UserProxyAgent, it transfers the control to the application/user, and waits for the feedback, once the feedback is provided, the control is transferred back to the team and the team continues its execution. 

# NOTE: When UserProxyAgent is called during a run, it blocks the execution of the team until the user provides feedback or errors out. This will hold up the team’s progress and put the team in an unstable state that cannot be saved or resumed.

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
# Create agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assitant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input) # Use input() to get user input from console.

#Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

In [3]:
# Create the team
team = RoundRobinGroupChat([assitant, user_proxy], termination_condition = termination)

In [4]:
# Run the conversation and stream the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")

await Console(stream)

await model_client.close()

---------- TextMessage (user) ----------
Write a 4-line poem about the ocean.
---------- TextMessage (assistant) ----------
Waves that dance in deep blue grace,  
Whispers of secrets the tides embrace.  
Beneath the sun, in endless motion,  
The heart of the world, the vast, deep ocean.  
TERMINATE
---------- TextMessage (user_proxy) ----------
APPROVE


# Using Max Turns \#
### This method allows you to pause the team for user input by setting a maximum number of turns. For instance, you can configure the team to stop after the first agent responds by setting max_turns to 1. This is particularly useful in scenarios where continuous user engagement is required, such as in a chatbot.
### To implement this, set the max_turns parameter in the RoundRobinGroupChat() constructor.

In [13]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent_max_turns = AssistantAgent("max_turns_agent", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
teams = RoundRobinGroupChat([agent_max_turns], max_turns=3)

In [14]:
task = "Write a 4-line poem about the ocean."
while True:
    # Run the conversation and stream the console.
    stream = teams.run_stream(task=task)
    await Console(stream)
    task = input("Enter your feedback (type 'exit' to leave):")
    if task.lower().strip() == "exit":
        break

await model_client.close()

---------- TextMessage (user) ----------
Write a 4-line poem about the ocean.
---------- TextMessage (max_turns_agent) ----------
Waves dance and shimmer under the sun's embrace,  
Endless horizon holds nature’s vast grace.  
Whispers of the deep tell tales of the tide,  
In the ocean's heart, all secrets abide.  
TERMINATE
---------- TextMessage (max_turns_agent) ----------
Waves dance and shimmer under the sun's embrace,  
Endless horizon holds nature’s vast grace.  
Whispers of the deep tell tales of the tide,  
In the ocean's heart, all secrets abide.  
TERMINATE
---------- TextMessage (max_turns_agent) ----------
Waves dance and shimmer under the sun's embrace,  
Endless horizon holds nature’s vast grace.  
Whispers of the deep tell tales of the tide,  
In the ocean's heart, all secrets abide.  
TERMINATE
---------- TextMessage (user) ----------
can you make it little funny
---------- TextMessage (max_turns_agent) ----------
The ocean's like a giant soup, all salty and blue,  
Fis

# Using Termination Condition \#
###  we focus on HandoffTermination which stops the team when an agent sends a HandoffMessage message.
### Let’s create a team with a single AssistantAgent agent with a handoff setting, and run the team with a task that requires additional input from the user because the agent doesn’t have relevant tools to continue processing the task.

In [21]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

In [22]:
load_dotenv()
# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

In [23]:
# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="If you cannot complete the task, transfer to user. Otherwise, when finished, respond with 'TERMINATE'.",
)

In [26]:
# Define a termination condition that checks for handoff messages.
handoff_termination = HandoffTermination(target="user")
# Define a termination condition that checks for a specific text mention.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team with the lazy assistant and both termination conditions.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=handoff_termination | text_termination)

# Run the team and stream to the console.
task = "What is the weather in New York?"
await Console(lazy_agent_team.run_stream(task=task), output_stats=True)
await Console(lazy_agent_team.run_stream(task="The weather in New York is sunny."))

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_MRl0zFAoNnna60wxQTHtLFdo', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 167, Completion tokens: 11]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_MRl0zFAoNnna60wxQTHtLFdo', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 167
Total completion tokens: 11
Duration: 1.11 seconds
---------- TextMessage (user) ----------
The weather in New York is sunny.
---------- TextMessage (lazy_assistant) ----------
Thank you for the information! If there's anything else you need, just let me know.
---------- ToolCallRequestEvent (lazy_assistant) ----------


TaskResult(messages=[TextMessage(id='0e36f0fd-7a98-4f99-95e6-279b192c3799', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 54, 927497, tzinfo=datetime.timezone.utc), content='The weather in New York is sunny.', type='TextMessage'), TextMessage(id='da9c831e-07ac-48f1-877c-657da62e0bd1', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=204, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 56, 131733, tzinfo=datetime.timezone.utc), content="Thank you for the information! If there's anything else you need, just let me know.", type='TextMessage'), ToolCallRequestEvent(id='59158220-1d90-4a43-b901-bf76ee1fb9d1', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=226, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 56, 986128, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_MLAxrkAO1QwZEACmgXiNdQE4', arguments='{}', name='transfer

In [25]:
await Console(lazy_agent_team.run_stream(task="The weather in New York is sunny."))


---------- TextMessage (user) ----------
The weather in New York is sunny.
---------- TextMessage (lazy_assistant) ----------
Thank you for the update! If you need any further information or assistance, feel free to ask.
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_Io9wFXSFFqCyZv5aqWbvoT1r', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_Io9wFXSFFqCyZv5aqWbvoT1r', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.


TaskResult(messages=[TextMessage(id='05a0db28-771a-4767-b414-a4e2f1a235b3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 0, 867236, tzinfo=datetime.timezone.utc), content='The weather in New York is sunny.', type='TextMessage'), TextMessage(id='ac5ec133-f1ec-49a0-87d1-05e73de06c87', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=106, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 2, 164695, tzinfo=datetime.timezone.utc), content='Thank you for the update! If you need any further information or assistance, feel free to ask.', type='TextMessage'), ToolCallRequestEvent(id='f9c46b6d-bacb-4677-a395-3fa70ceecf05', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=130, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 8, 4, 17, 42, 3, 490740, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_Io9wFXSFFqCyZv5aqWbvoT1r', arguments='{}', name='